In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [2]:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# import seaborn as sns
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures

In [4]:
df = pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")
# df.head()

In [ ]:
df_test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")
df_test.head()

In [ ]:
df.info()

In [ ]:
df.describe().round(1)

In [6]:
df.isna().sum()

id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

In [ ]:
sns.heatmap(df.isna().sort_values(by=["Occupation"]) == True)

In [5]:
# df.dropna(inplace = True)
for non_object_column in df.select_dtypes(exclude = "object").columns:
    df[non_object_column].fillna(df[non_object_column].mean(), inplace = True)

df.dropna(inplace = True)

In [7]:
df.shape

(775274, 21)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(df.select_dtypes(exclude = "object").corr(), annot = True, cmap = "Reds")

In [ ]:
df_categorical = df.select_dtypes('object')
for object_column in df_categorical.columns:
    print(object_column, {df[object_column].nunique()}, ': ', df[object_column].unique(), '\n')

In [6]:
df.drop(['id', 'Policy Start Date'], axis = 1, inplace = True)

In [ ]:
for non_object_column in df.select_dtypes(exclude = "object").columns:
    if non_object_column != 'Premium Amount':
        df[non_object_column] = df[non_object_column] / df[non_object_column].max()

In [ ]:
df.head()

In [ ]:
df["Policy Start Date"] = pd.to_datetime(df["Policy Start Date"].apply(lambda x: x[:10]))

In [ ]:
pd.get_dummies(df_categorical, drop_first = True, dtype = int)

In [7]:
df = pd.concat(
    [
        df.select_dtypes(exclude = "object"), 
        pd.get_dummies(df.select_dtypes('object'), drop_first = True, dtype = int)
    ],
    axis = "columns"
)
df.head()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount,Gender_Male,...,Policy Type_Comprehensive,Policy Type_Premium,Customer Feedback_Good,Customer Feedback_Poor,Smoking Status_Yes,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Property Type_Condo,Property Type_House
0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.00000,5.0,2869.0,0,...,0,1,0,1,0,0,0,1,0,1
2,23.0,25602.0,3.0,47.177549,1.0,14.0,592.92435,3.0,567.0,1,...,0,1,1,0,1,0,0,1,0,1
4,21.0,39651.0,1.0,20.376094,0.0,8.0,598.00000,4.0,2022.0,1,...,0,1,0,1,1,0,0,1,0,1
7,48.0,127237.0,2.0,5.769783,1.0,11.0,398.00000,5.0,111.0,0,...,1,0,0,0,0,0,1,0,1,0
9,44.0,52447.0,2.0,20.473718,1.0,9.0,635.00000,3.0,64.0,1,...,1,0,0,1,0,0,0,0,1,0


In [ ]:
df.dtypes

In [8]:
x = df.drop(['Premium Amount'], axis = 1)
y = df['Premium Amount']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [10]:
model1 = LinearRegression()
# model2 = RandomForestRegressor(n_estimators=28, random_state=0)
model3 = PolynomialFeatures(degree=4)

In [ ]:
x_train = model3.fit_transform(x_train)
x_test = model3.fit_transform(x_test)
x = model3.fit_transform(x)

In [13]:
model = model3

model.fit(x_train, y_train)

model = model1
model.fit(x_train, y_train)

PolynomialFeatures(degree=4)

In [48]:
y_pred = model.predict(x_test)

AttributeError: 'PolynomialFeatures' object has no attribute 'predict'

In [37]:
def rmsle(predictions, targets):
    """Calculate the root mean squared logarithmic error between predictions and targets"""
    return np.sqrt(np.mean((np.log(predictions + 1) - np.log(targets + 1)) ** 2))

In [38]:
rmsle(y_pred, y_test)

1.1598902905318074

In [39]:
model .fit(x, y)

RandomForestRegressor(n_estimators=28, random_state=0)

In [40]:
x.shape

(775274, 28)

In [41]:
df_test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")

# for non_object_column in df_test.select_dtypes(exclude = "object").columns:
#     df_test[non_object_column] = df_test[non_object_column] / df_test[non_object_column].max()

for non_object_column in df_test.select_dtypes(exclude = "object").columns:
    df_test[non_object_column].fillna(df_test[non_object_column].mean(), inplace = True)

df_test["Policy Start Date"] = pd.to_datetime(df_test["Policy Start Date"].apply(lambda x: x[:10]))

df_test = pd.concat(
    [
        df_test.select_dtypes(exclude = "object"), 
        pd.get_dummies(df_test.select_dtypes('object'), drop_first = True, dtype = int)
    ],
    axis = "columns"
)

x_df_test = df_test.drop(['id', 'Policy Start Date'], axis = 1)

print(x_df_test.shape)
x_df_test.head()

(800000, 28)


,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Gender_Male,Marital Status_Married,...,Policy Type_Comprehensive,Policy Type_Premium,Customer Feedback_Good,Customer Feedback_Poor,Smoking Status_Yes,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Property Type_Condo,Property Type_House
0,28.0,2310.0,4.0,7.657981,1.004873,19.0,592.904749,1.0,0,0,...,0,0,0,1,1,0,0,1,0,1
1,31.0,126031.0,2.0,13.381379,1.004873,14.0,372.000000,8.0,0,1,...,0,1,1,0,1,0,1,0,0,0
2,47.0,17092.0,0.0,24.354527,1.004873,16.0,819.000000,9.0,0,0,...,1,0,0,0,1,1,0,0,1,0
3,28.0,30424.0,3.0,5.136225,1.000000,3.0,770.000000,5.0,0,0,...,1,0,0,1,1,0,0,0,0,1
4,24.0,10863.0,2.0,11.844155,1.004873,14.0,755.000000,7.0,1,0,...,0,1,0,0,0,0,0,1,0,1


In [42]:
Y_pred_test = model.predict(x_df_test)

In [ ]:
df_sub = pd.read_csv("/kaggle/input/playground-series-s4e12/sample_submission.csv")
df_sub.head()

In [43]:
df_test['Premium Amount'] = Y_pred_test.round(3)
df_submission = df_test[['id', 'Premium Amount']]

print(df_submission.shape)
df_submission.head()

(800000, 2)


,id,Premium Amount
0,1200000,1707.929
1,1200001,1187.571
2,1200002,1301.286
3,1200003,1114.607
4,1200004,906.536


In [44]:
df_submission.to_csv("submissions.csv", index=False)

In [47]:
df_test.isna().sum()

id                                  0
Age                             12489
Annual Income                   29860
Number of Dependents            73130
Health Score                    49449
Previous Claims                242802
Vehicle Age                         3
Credit Score                    91451
Insurance Duration                  2
Policy Start Date                   0
Gender_Male                         0
Marital Status_Married              0
Marital Status_Single               0
Education Level_High School         0
Education Level_Master's            0
Education Level_PhD                 0
Occupation_Self-Employed            0
Occupation_Unemployed               0
Location_Suburban                   0
Location_Urban                      0
Policy Type_Comprehensive           0
Policy Type_Premium                 0
Customer Feedback_Good              0
Customer Feedback_Poor              0
Smoking Status_Yes                  0
Exercise Frequency_Monthly          0
Exercise Fre